<a href="https://colab.research.google.com/github/YoonHanho/coin_trading/blob/main/my_upbit_scoring_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 코인 스코어링(25점 만점)

1. 최근 5일 가격 증가 : 일자별 5점
2. 최근 5일 거래량 증가 : 일자별 5점 
3. 최근일 범위 초과 상승 : 범위별 5점 
4. 당일 코인 거래대금 순위 : 순위별 5점 
5. 주가 정배열 : 5일이평/20일이평 대비 모두 상승 5점, 5일이평 대비 상승 3점. 나머지 0점

- https://wikidocs.net/31063

In [17]:
!pip install pyupbit

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

import pyupbit
import pprint

# 조회만 되는 키 
access_key = "HERE'S YOUR KEY"
secret_key = "HERE'S YOUR KEY"

upbit = pyupbit.Upbit(access_key, secret_key)

In [3]:
# 과거 데이터
df = pyupbit.get_ohlcv("KRW-MTL")
df.tail(6)

,open,high,low,close,volume,value
2022-04-29 09:00:00,3485.0,3515.0,3280.0,3335.0,1.924058e+07,6.531214e+10
2022-04-30 09:00:00,3330.0,3350.0,2720.0,2790.0,1.598971e+07,4.875600e+10
2022-05-01 09:00:00,2790.0,2845.0,2510.0,2660.0,1.577174e+07,4.166636e+10
2022-05-02 09:00:00,2660.0,2890.0,2535.0,2670.0,2.975040e+07,8.117859e+10
2022-05-03 09:00:00,2665.0,2670.0,2445.0,2485.0,9.842606e+06,2.516083e+10
2022-05-04 09:00:00,2485.0,2505.0,2320.0,2435.0,9.348915e+06,2.252656e+10


In [6]:
def get_coin_score(ticker):

  df = pyupbit.get_ohlcv(ticker)
  h = 21    # 현재시간 강제할당(Colab서버 --> 미국시간) : h = datetime.now().hour

  # 최근 5일 상승여부에 따라 일자별 5점 (close, volume) 
  def get_5days(feat):
    if feat == 'volume':    # 거래량의 경우, 당일은 현재 시간을 24시간으로 보정 필요
      df['volume'].iloc[-1] = df['volume'].iloc[-1] / max(1, h) * 24
    ser = df[feat] - df[feat].shift()
    res = np.where(ser > 0, 1, 0)[-5:]    
    return res.sum()
        
  #전일가격범위 대비 하여 시초가 가격이 오른 비율
  def get_range():
    range = (df.iloc[-2]['high'] - df.iloc[-2]['low']) 
    today_range = ( df['close'].iloc[-1] - df['open'].iloc[-1] ) / range
    
    try: 
      today_range = ( df['close'].iloc[-1] - df['open'].iloc[-1] ) / range

      if today_range < 0.1:
        range_score = 0
      elif today_range < 0.2:
        range_score = 1
      elif today_range < 0.3:
        range_score = 2
      elif today_range < 0.4:
        range_score = 3
      elif today_range < 0.5:
        range_score = 4
      elif today_range >= 0.5:
        range_score = 5
    except:
      print("가격범위 산정 오류(0값 부여) : ", ticker)
      range_score = 0

    return range_score

  def get_trade():
    return df['value'][-1]

  def price_arrange():
    ma5 = df['close'].rolling(window=5).mean()[-2]
    ma20 = df['close'].rolling(window=20).mean()[-2]
    price = pyupbit.get_current_price(ticker)

    if price > ma5 > ma20:
      return 5
    elif price > ma5 :
      return 3
    else:
      return 0

  price_score = get_5days('close')
  volume_score = get_5days('volume')
  range_score = get_range()
  trade_score = get_trade()
  arrange_score = price_arrange()

  return price_score, volume_score, range_score, trade_score, arrange_score

In [7]:
get_coin_score('KRW-BTC')

(3, 2, 5, 82788074096.4395, 3)

In [8]:
tickers = pyupbit.get_tickers(fiat="KRW")

coin_score = []
for ticker in tickers:
  try:
    p, v, r, t, a = get_coin_score(ticker)
    coin_score.append([ticker, p, v, r, t, a])
  except:
    print("실행오류 : ", ticker)
    pass 
  time.sleep(0.5)

실행오류 :  KRW-GMT


In [10]:
res = pd.DataFrame(coin_score)
res.columns = ['ticker', 'price', 'volume', 'range', 'trade', 'arrange']

# 거래금액 상대 순위 점수 
res['trade'] = (res['trade'].rank(pct=True) * 5).astype(int) 

# 총점 
res['total_score'] = res.iloc[:,1:].sum(axis=1)

# 최종 결과 파일
score_final = res.sort_values(by='total_score', ascending=False).reset_index(drop=True)

순위별 종목 추출

In [11]:
# 상위 10종목
score_final.iloc[:10]

,ticker,price,volume,range,trade,arrange,total_score
0,KRW-TRX,3,4,5,5,5,22
1,KRW-SAND,3,3,5,4,3,18
2,KRW-WAVES,2,4,5,4,3,18
3,KRW-ZRX,3,3,5,4,3,18
4,KRW-NEAR,4,2,5,4,3,18
5,KRW-ETH,3,2,5,4,3,17
6,KRW-ICX,3,4,3,4,3,17
7,KRW-ADA,2,3,5,4,3,17
8,KRW-BTC,3,2,5,4,3,17
9,KRW-ETC,2,2,5,4,3,16


In [12]:
# 하위 10종목 
score_final.iloc[-10:]

,ticker,price,volume,range,trade,arrange,total_score
103,KRW-MVL,1,1,4,0,0,6
104,KRW-GLM,2,2,1,0,0,5
105,KRW-RFR,1,3,1,0,0,5
106,KRW-ARDR,1,2,0,2,0,5
107,KRW-KAVA,1,1,0,3,0,5
108,KRW-ORBS,2,2,1,0,0,5
109,KRW-CRE,2,2,1,0,0,5
110,KRW-GRS,2,1,1,0,0,4
111,KRW-NU,0,1,0,3,0,4
112,KRW-IQ,2,2,0,0,0,4


관심종목 추출

In [16]:
score_final.query("ticker == 'KRW-BTC' ")  # 비트코인

,ticker,price,volume,range,trade,arrange,total_score
8,KRW-BTC,3,2,5,4,3,17


In [15]:
score_final.query("ticker == 'KRW-ETH' ")  #  이더리움

,ticker,price,volume,range,trade,arrange,total_score
5,KRW-ETH,3,2,5,4,3,17
